### Imports

In [3]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))
import constants, utils

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage

os.environ["OPENAI_API_KEY"] = utils.get_openai_api_key()
os.environ["TAVILY_API_KEY"] = utils.get_tavily_api_key()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Search Tool Definition

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1718279466, 'localtime': '2024-06-13 4:51'}, 'current': {'last_updated_epoch': 1718279100, 'last_updated': '2024-06-13 04:45', 'temp_c': 13.9, 'temp_f': 57.0, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 340, 'wind_dir': 'NNW', 'pressure_mb': 1012.0, 'pressure_in': 29.89, 'precip_mm': 0.01, 'precip_in': 0.0, 'humidity': 87, 'cloud': 75, 'feelslike_c': 12.5, 'feelslike_f': 54.4, 'windchill_c': 8.9, 'windchill_f': 48.0, 'heatindex_c': 11.0, 'heatindex_f': 51.8, 'dewpoint_c': 8.9, 'dewpoint_f': 48.0, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 16.6, 'gust_kph': 26.7}}"}, {'url': 'https://www.wunderground.com/hourly/

### Using a Language Model

In [5]:
model = ChatOpenAI(model=constants.MODEL_NAME)

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [7]:
model_with_tools = model.bind_tools(tools)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [8]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_9V3bZg42tBaS4lKvvOuEHMQn'}]


### Create and Run the Agent

In [12]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [13]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='5fd100e0-d76a-437e-88a9-04a6e00ec8b9'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 83, 'total_tokens': 93}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-03d6b4fd-6025-428a-90c6-374813a3360f-0', usage_metadata={'input_tokens': 83, 'output_tokens': 10, 'total_tokens': 93})]

In [14]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='6350d2f3-e18c-4aad-bfe5-02e7ee579df6'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_AOSKlG76P36uUTAwuCHGA3mA', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-578d75ea-7ac3-477d-99e7-339ad91e3072-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_AOSKlG76P36uUTAwuCHGA3mA'}], usage_metadata={'input_tokens': 88, 'output_tokens': 21, 'total_tokens': 109}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37

### Streaming Agent Execution Process

In [15]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_MBYTSBhorB4OfIiyS00A818v', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 88, 'total_tokens': 109}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-af6b1cfc-42c7-44a3-b2af-7a2052767042-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_MBYTSBhorB4OfIiyS00A818v'}], usage_metadata={'input_tokens': 88, 'output_tokens': 21, 'total_tokens': 109})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'Ameri

### Adding Memory

In [17]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 85, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2ffa73fd-82e4-4ce3-bf63-287af9d7f973-0', usage_metadata={'input_tokens': 85, 'output_tokens': 11, 'total_tokens': 96})]}}
----


In [18]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob! How can I help you further, Bob?', response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 108, 'total_tokens': 123}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-256ce44b-7d83-4bd7-9c03-386ac4775419-0', usage_metadata={'input_tokens': 108, 'output_tokens': 15, 'total_tokens': 123})]}}
----
